In [1]:
import csv

# ============================================================
#                  GLOBAL STORAGE
# ============================================================

CUSTOMERS = []   # Global list of Customer objects


# ============================================================
#                    CLASS DEFINITIONS
# ============================================================

class Account:
    """
    Abstract Account class.
    
    Attributes
    ----------
    account_id : str
        Four-digit numeric string, e.g. '1234'
    balance : float
        Account balance (>= 0 for Checking/Savings; >=0 charges for Credit)
    interest : float
        Interest rate (0 for Checking, 1 for Savings, 30 for Credit)
    """
    
    def __init__(self, account_id, balance, interest=0.0):
        self.set_account_id(account_id)
        self.set_balance(balance)
        self.set_interest(interest)

    # ---------------- GET METHODS ----------------
    def get_account_id(self):
        return self._account_id

    def get_balance(self):
        return self._balance

    def get_interest(self):
        return self._interest

    # ---------------- SET METHODS ----------------
    def set_account_id(self, account_id):
        if not isinstance(account_id, str):
            raise TypeError("account_id must be a string.")
        if len(account_id) != 4:
            raise ValueError("account_id must be a four-digit string.")
        if any(ch not in "0123456789" for ch in account_id):
            raise ValueError("account_id must contain only digits.")
        self._account_id = account_id

    def set_balance(self, balance):
        if not isinstance(balance, (int, float)):
            raise TypeError("balance must be numeric.")
        self._balance = float(balance)

    def set_interest(self, interest):
        if not isinstance(interest, (int, float)):
            raise TypeError("interest must be numeric.")
        self._interest = float(interest)

    # ---------------- METHODS ----------------
    def deposit(self, amount):
        """Deposit into Checking or Savings (Credit not allowed)."""
        if amount < 0:
            raise ValueError("Cannot deposit a negative amount.")
        self._balance += amount

    def withdraw(self, amount):
        """Withdraw from Checking or Savings with balance check."""
        if amount < 0:
            raise ValueError("Cannot withdraw a negative amount.")
        if self._balance - amount < 0:
            raise ValueError("Withdrawal denied: insufficient funds.")
        self._balance -= amount

    # ---------------- STRING ----------------
    def __str__(self):
        return f"Account {self._account_id}: Balance ${self._balance:.2f}, Interest {self._interest}%"


class Checking(Account):
    """Checking account with 0% interest."""
    def __init__(self, account_id, balance):
        super().__init__(account_id, balance, interest=0.0)

    def __str__(self):
        return f"Checking {self.get_account_id()}: Balance ${self.get_balance():.2f}"


class Savings(Account):
    """Savings account with 1% interest."""
    def __init__(self, account_id, balance):
        super().__init__(account_id, balance, interest=1.0)

    def __str__(self):
        return f"Savings {self.get_account_id()}: Balance ${self.get_balance():.2f}"


class Credit(Account):
    """
    Credit account with additional credit_limit attribute.
    
    balance = total charges
    """
    def __init__(self, account_id, balance, credit_limit):
        super().__init__(account_id, balance, interest=30.0)
        self.set_credit_limit(credit_limit)

    # GET/SET
    def get_credit_limit(self):
        return self._credit_limit

    def set_credit_limit(self, credit_limit):
        if not isinstance(credit_limit, (int, float)):
            raise TypeError("credit_limit must be numeric.")
        if credit_limit < 0:
            raise ValueError("credit_limit cannot be negative.")
        self._credit_limit = float(credit_limit)

    # METHODS
    def charge(self, amount):
        """Increase balance (charges)."""
        if amount < 0:
            raise ValueError("Cannot charge a negative amount.")
        if self._balance + amount > self._credit_limit:
            raise ValueError("Charge denied: exceeds credit limit.")
        self._balance += amount

    def pay(self, amount, paying_account):
        """
        Pay credit card bill using Checking or Savings.
        paying_account is a Checking or Savings object.
        """
        if amount < 0:
            raise ValueError("Cannot pay a negative amount.")
        if amount > self._balance:
            raise ValueError("Payment denied: amount exceeds credit balance.")
        if paying_account.get_balance() < amount:
            raise ValueError("Payment denied: insufficient funds in paying account.")

        paying_account.withdraw(amount)
        self._balance -= amount

    # STRING
    def __str__(self):
        return (f"Credit {self.get_account_id()}: Balance ${self.get_balance():.2f}, "
                f"Limit ${self.get_credit_limit():.2f}")


class Customer:
    """Customer with username and three accounts."""
    def __init__(self, username, checking, savings, credit):
        self.set_username(username)
        self.set_checking(checking)
        self.set_savings(savings)
        self.set_credit(credit)

    # GET/SET
    def get_username(self):
        return self._username

    def set_username(self, username):
        if not isinstance(username, str):
            raise TypeError("username must be a string.")
        self._username = username

    def get_checking(self):
        return self._checking

    def set_checking(self, account):
        if not isinstance(account, Checking):
            raise TypeError("checking must be a Checking object.")
        self._checking = account

    def get_savings(self):
        return self._savings

    def set_savings(self, account):
        if not isinstance(account, Savings):
            raise TypeError("savings must be a Savings object.")
        self._savings = account

    def get_credit(self):
        return self._credit

    def set_credit(self, account):
        if not isinstance(account, Credit):
            raise TypeError("credit must be a Credit object.")
        self._credit = account

    def __str__(self):
        return (f"Customer {self._username}\n"
                f"  {self._checking}\n"
                f"  {self._savings}\n"
                f"  {self._credit}")


# ============================================================
#                UTILITY / LOOKUP FUNCTIONS
# ============================================================

def lookup_customer(username):
    """Return customer object with matching username."""
    for c in CUSTOMERS:
        if c.get_username() == username:
            return c
    return None


# ============================================================
#                  FILE IMPORT / EXPORT
# ============================================================

def import_customers():
    filename = input("Enter CSV filename to import: ").strip()

    try:
        with open(filename, newline='') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header

            for row in reader:
                username, chk_id, chk_bal, sav_id, sav_bal, cred_id, cred_bal, cred_limit = row

                checking = Checking(chk_id, float(chk_bal))
                savings = Savings(sav_id, float(sav_bal))
                credit = Credit(cred_id, float(cred_bal), float(cred_limit))

                customer = Customer(username, checking, savings, credit)
                CUSTOMERS.append(customer)

        print("Import successful.\n")

    except Exception as e:
        print("Import failed:", e)


def export_customers():
    filename = input("Enter CSV filename to export: ").strip()

    try:
        with open(filename, "w", newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                "username", "checking_id", "checking_balance",
                "savings_id", "savings_balance",
                "credit_id", "credit_balance", "credit_limit"
            ])

            for cust in CUSTOMERS:
                writer.writerow([
                    cust.get_username(),
                    cust.get_checking().get_account_id(),
                    f"{cust.get_checking().get_balance():.2f}",
                    cust.get_savings().get_account_id(),
                    f"{cust.get_savings().get_balance():.2f}",
                    cust.get_credit().get_account_id(),
                    f"{cust.get_credit().get_balance():.2f}",
                    f"{cust.get_credit().get_credit_limit():.2f}"
                ])

        print("Export successful.\n")

    except Exception as e:
        print("Export failed:", e)


# ============================================================
#                       OPERATIONS
# ============================================================

def view_customers():
    for c in CUSTOMERS:
        print(c)
        print()


def deposit():
    username = input("Customer username: ").strip()
    customer = lookup_customer(username)
    if not customer:
        print("Customer not found.")
        return

    acct_type = input("Deposit to (checking/savings): ").lower()
    amount = float(input("Amount: "))

    try:
        if acct_type == "checking":
            customer.get_checking().deposit(amount)
        elif acct_type == "savings":
            customer.get_savings().deposit(amount)
        else:
            print("Invalid account type.")
    except Exception as e:
        print(e)


def withdraw():
    username = input("Customer username: ").strip()
    customer = lookup_customer(username)
    if not customer:
        print("Customer not found.")
        return

    acct_type = input("Withdraw from (checking/savings): ").lower()
    amount = float(input("Amount: "))

    try:
        if acct_type == "checking":
            customer.get_checking().withdraw(amount)
        elif acct_type == "savings":
            customer.get_savings().withdraw(amount)
        else:
            print("Invalid account type.")
    except Exception as e:
        print(e)


def credit_charge():
    username = input("Customer username: ").strip()
    customer = lookup_customer(username)
    if not customer:
        print("Customer not found.")
        return

    amount = float(input("Charge amount: "))

    try:
        customer.get_credit().charge(amount)
    except Exception as e:
        print(e)


def credit_payment():
    username = input("Customer username: ").strip()
    customer = lookup_customer(username)
    if not customer:
        print("Customer not found.")
        return

    acct_type = input("Pay using (checking/savings): ").lower()
    amount = float(input("Payment amount: "))

    try:
        if acct_type == "checking":
            paying = customer.get_checking()
        elif acct_type == "savings":
            paying = customer.get_savings()
        else:
            print("Invalid account type.")
            return

        customer.get_credit().pay(amount, paying)

    except Exception as e:
        print(e)


# ============================================================
#                     MAIN INTERFACE
# ============================================================

def interface():
    while True:
        print("\n===== MAIN MENU =====")
        print("1. Import")
        print("2. View Customers")
        print("3. Deposit")
        print("4. Withdraw")
        print("5. Credit Card Charge")
        print("6. Credit Card Payment")
        print("7. Exit (Export)")

        choice = input("Select an option: ").strip()

        if choice == "1":
            import_customers()
        elif choice == "2":
            view_customers()
        elif choice == "3":
            deposit()
        elif choice == "4":
            withdraw()
        elif choice == "5":
            credit_charge()
        elif choice == "6":
            credit_payment()
        elif choice == "7":
            export_customers()
            print("Goodbye!")
            break
        else:
            print("Invalid option. Try again.")